数据的预处理部分

In [32]:
import os 
import shutil
import numpy as np 
import time

root_src='D:\workfile\python code\projects\dog_vs_cat_MLND'
os.chdir(root_src)

def create_directory(dir_name):
    if os.path.exists(dir_name):
        pass
    else:
        os.mkdir(dir_name)

create_directory('train2')
create_directory('train2/cat')
create_directory('train2/dog')


import random

filenames=os.listdir('train/train/')
cat_file=list(filter(lambda x:x[:3] =='cat', filenames))
cat_file_num=len(cat_file)

dog_file=list(filter(lambda x:x[:3] =='dog', filenames))
dog_file_num=len(dog_file)

print("the cats are %d, dogs are %d"%(cat_file_num, dog_file_num))

for name in cat_file:
    shutil.move('train/train/'+name, 'train2/cat/'+name)
for name in dog_file:
    shutil.move('train/train/'+name, 'train2/dog/'+name)


filenames=os.listdir('test1/test1')

print("the test is %d" %len(filenames))

the cats are 0, dogs are 0
the test is 12500


这里为了减少模型训练的时间，有效利用前人的经验提高准确率，选择了4个训练好的模型，ResNet50,
Xception，InceptionV3 和VGG19。这四个都是前人总结发展的非常好的模型，可以帮助我们高速有效的
构建一个深度学习分类器

载入这些特征向量，将它们合并成一条特征向量，随机打乱X,y。

In [33]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train=[]
X_test=[]

for filename in ['gap_VGG19.h5', 'gap_Xception.h5', 'gap_ResNet50.h5','gap_InceptionV3.h5']:
    with h5py.File(filename, 'r') as h:
        print(h.keys())
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train=np.array(h['label'])
        
X_train=np.concatenate(X_train, axis=1)
X_test=np.concatenate(X_test, axis=1)

X_train, y_train=shuffle(X_train, y_train)

KeysView(<HDF5 file "gap_VGG19.h5" (mode r)>)
KeysView(<HDF5 file "gap_Xception.h5" (mode r)>)
KeysView(<HDF5 file "gap_ResNet50.h5" (mode r)>)
KeysView(<HDF5 file "gap_InceptionV3.h5" (mode r)>)


构建模型

In [34]:
from keras.models import *
from keras.layers import *
np.random.seed(2017)

input_tensor=Input(X_train.shape[1:])
x=Dropout(0.5)(input_tensor)
x=Dense(1,activation='sigmoid')(x)

model=Model(input_tensor, x)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])


模型的训练

In [35]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s - loss: 0.1772 - acc: 0.9305 - val_loss: 0.0266 - val_acc: 0.9910
Epoch 2/10
20000/20000 [==============================] - 1s - loss: 0.0430 - acc: 0.9846 - val_loss: 0.0171 - val_acc: 0.9944
Epoch 3/10
20000/20000 [==============================] - 1s - loss: 0.0286 - acc: 0.9909 - val_loss: 0.0160 - val_acc: 0.9940
Epoch 4/10
20000/20000 [==============================] - 1s - loss: 0.0260 - acc: 0.9913 - val_loss: 0.0132 - val_acc: 0.9952
Epoch 5/10
20000/20000 [==============================] - 1s - loss: 0.0217 - acc: 0.9931 - val_loss: 0.0125 - val_acc: 0.9956
Epoch 6/10
20000/20000 [==============================] - 1s - loss: 0.0201 - acc: 0.9937 - val_loss: 0.0120 - val_acc: 0.9958
Epoch 7/10
20000/20000 [==============================] - 1s - loss: 0.0188 - acc: 0.9938 - val_loss: 0.0115 - val_acc: 0.9962
Epoch 8/10
20000/20000 [==============================] - 1s -

预测训练集

In [22]:
y_pred=model.predict(X_test, verbose=1)
y_pred=y_pred.clip(min=0.005, max=0.995)

import pandas as pd 
from keras.preprocessing.image import *
df=pd.read_csv('sample_submission.csv')

gen=ImageDataGenerator()

test_generator=gen.flow_from_directory('test1', (224,224), shuffle=False, batch_size=32, class_mode=None)
for i, fname in enumerate(test_generator.filenames):
    index=int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    index=index-1.0
    df.set_value(index, 'label', y_pred[i])
df.to_csv('pred.csv', index=None)
df.head(10)

11168/12500 [=========================>....] - ETA: 0sFound 12500 images belonging to 1 classes.


D:\program\Anaconda\envs\dogcat\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


总结：我们使用和四种预训练模型，结果还不错。 